In [1]:
import os

#check and set working directory
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory

os.chdir('c:\\Users\\j_m289\\Pictures\\PHD\\15. Data Analysis\HMR\Working')  # Directory set to 'clean' folder

#try to use relative path for files (tied to directory and nesting within folders)
path = "/Users/j_m289\Pictures\PHD\15. Data Analysis\HMR\Working"
  
start = "/Users/j_m289"

relative_path = os.path.relpath(path, start)

print(cwd)

#Load packages
import pandas as pd #pandas for data frame management and descriptives (in addition to numpy)
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import chi2
import xlsxwriter

c:\Users\j_m289\Pictures\PHD\15. Data Analysis\HMR\Code


In [ ]:
#Use this one! Update with flow for removing multivariate outliers

# Read completed dataset with scales for each diagnosis sub-type spreadsheet from 'Working' folder. 
def outlierdetection(df, threshold = 3):
    # create an empty DataFrame to store the results
    results = pd.DataFrame()
    writer = pd.ExcelWriter('outlier_results.xlsx', engine='xlsxwriter')

    # loop over each column in the DataFrame
    for col in df.columns:
        # calculate the median for the column
        median = df[col].median()

        # calculate the median absolute deviation for the column
        mad = df[col].mad()

        # calculate the lower and upper bounds for outliers
        lower_bound = median - (threshold * mad)
        upper_bound = median + (threshold * mad)
        data_between_bounds = df[col].between(lower_bound, upper_bound)
        outliers = data_between_bounds[data_between_bounds == False].index.tolist()
        n_outliers = len(outliers)
        
        # Plot to visualise outliers 
        ax = sns.boxplot(data=df[col])
        ax.set_title(f"Outliers in Column '{col}'")
        sns.scatterplot(x=np.where(outliers)[0] + 1, y=df.loc[outliers, col], color='red', label='Outliers')
        for i, outlier in enumerate(df.loc[outliers, col]):
            plt.annotate(str(i+1), (np.where(outliers)[0][i] + 1, outlier))
        plt.show()

        # ask the user whether to drop the outliers
        drop_outliers = input(f"Do you want to drop the outliers for column {col}? (y/n): ")

        # If response = y, then:
        if drop_outliers.lower() == 'y':
            df = df.drop(outliers)
        pass

        # add the results to the results DataFrame
        results[col] = pd.Series({'Median': median,
                                   'MAD': mad,
                                   'Lower': lower_bound,
                                   'Upper': upper_bound,
                                   'Outliers': outliers, #.any(),
                                   'Num_outliers': n_outliers})
        
        # save results to a separate sheet in the Excel file
        results.to_excel(writer, sheet_name=col, index=True)

    # save the Excel file
    writer.save()

    return results, df

# create a sample DataFrame
df = pd.read_csv('AnorexiaDataset.csv') # Anorexia Nervosa
test = ['BMI', 'YPSQ_EFSA']
variables = ['Age','BMI','YPSQ_EFSA','YPSQ_S','YPSQ_EC','YPSQ_O','YPSQ_EOS',\
    'YPSQ_SCRE','YPSQ_DS','YPSQ_SB','YPSQ_HSC','EmReg_CR','EmReg_ES','EDE_Q_R','EDE_Q_W','EDE_Q_S','EDE_Q_E', 'EDE_Q_G','CFI_Alt', 'CFI_Ctrl']
# call the outlierdetection function
results, df_no_outliers = outlierdetection(df[variables]) # this is multiple assignment and assigns the return of the function to the objects

# print the results
display (results)
print('\nDataframe with outliers removed:\n', df_no_outliers)


# Checkpoint 4 - Outlier detection and analysis dataset export
1. Check univariate and multivariate outliers per diagnosis subtype (as they are analysed independently)
2. Report the number of cases (and ID's) excluded based on outlier detection
3. Save dataframes containing only the composite scores/variables of interest
4. Save dataframes to analysis folder for HMR analysis and assumption checks in SPSS

In [14]:
# Read completed dataset with scales for each diagnosis sub-type spreadsheet from 'Working' folder. 
# Save datasets to dataframes
df_ED_AN = pd.read_csv('AnorexiaDataset.csv', index_col=[0]) # Anorexia Nervosa
df_ED_BN = pd.read_csv('BulimiaDataset.csv', index_col=[0]) # Bulimia Nervosa
df_ED_BED = pd.read_csv('BingeEatingDataset.csv', index_col=[0]) # Binge Eating Disorder
df_ED_OSFED = pd.read_csv('OsfedDataset.csv', index_col=[0]) # OSFED
df_healthy = pd.read_csv('HealthyDataset.csv', index_col=[0]) # Healthy

pd.set_option("display.max_rows", 10) #adjust max rows displayed as needed

# Define functions
def uni_outliers(data, name, threshold=3): #Function to determine univariate outliers (draft). 
    print('Variable:', name)
    median = np.median(data)
    mad = np.median([np.abs(y - median) for y in data])*1.4826
    lower_bound = median - (threshold * mad)
    upper_bound = median + (threshold * mad)
    print('Median:',median,'\nMAD:',mad,'\nLower bound:',lower_bound,'\nUpper bound:',upper_bound)
    data_between_bounds = data.between(lower_bound, upper_bound)
    idx_bounds = data_between_bounds[data_between_bounds == False].index.tolist()
    return idx_bounds
 
def mahalanobis(x=None, data=None, cov=None): #Function to determine multivariate outliers (draft).
    """Calculate the Mahalanobis Distance between each row of x and the data  
    x : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be calculated.
    cov : covariance matrix (p x p) of the distribution. If None, will be calculated from data.
    """
    x_minus_mu = x - np.mean(data)
    if not cov:
        cov = np.cov(data.values.T)
    inv_covmat = np.linalg.inv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()

# work in progress
# def outliers (data, name):
   # for x in data:
    #   uni_outliers(data, name, threshold=2.5)
    #  if idx_bounds 

# Set variables containing columns to pass into functions
test = ['BMI']

variables = ['Age','BMI','YPSQ_EFSA','YPSQ_S','YPSQ_EC','YPSQ_O','YPSQ_EOS',\
    'YPSQ_SCRE','YPSQ_DS','YPSQ_SB','YPSQ_HSC','EmReg_CR','EmReg_ES','EDE_Q_R','EDE_Q_W','EDE_Q_S','EDE_Q_E', 'EDE_Q_G','CFI_Alt', 'CFI_Ctrl']

In [15]:
#Univariate Outliers - Anorexia Nervosa 
AN_uni_vars = [None]*len(variables)
for i, x in enumerate(variables): #creates two variables for loop (i = 0, 1, 2.....)
    AN_uni_vars[i] = uni_outliers(df_ED_AN[x], x, threshold=3)
    print('Outlier Indices:', AN_uni_vars[i], '\n ---------')

Variable: Age
Median: 24.0 
MAD: 5.9304 
Lower bound: 6.2088 
Upper bound: 41.7912
Outlier Indices: [622, 693, 934, 964, 980, 1078, 1130, 1347] 
 ---------
Variable: BMI
Median: 18.548979397026386 
MAD: 2.935351237277821 
Lower bound: 9.742925685192922 
Upper bound: 27.35503310885985
Outlier Indices: [471, 580, 883, 1066, 1094, 1126, 1236, 1262, 1438, 1454, 1549] 
 ---------
Variable: YPSQ_EFSA
Median: 2.833333333333333 
MAD: 1.4826 
Lower bound: -1.614466666666667 
Upper bound: 7.281133333333333
Outlier Indices: [] 
 ---------
Variable: YPSQ_S
Median: 4.2 
MAD: 1.1860799999999996 
Lower bound: 0.6417600000000014 
Upper bound: 7.758239999999999
Outlier Indices: [] 
 ---------
Variable: YPSQ_EC
Median: 4.333333333333333 
MAD: 0.9884000000000004 
Lower bound: 1.3681333333333319 
Upper bound: 7.298533333333334
Outlier Indices: [] 
 ---------
Variable: YPSQ_O
Median: 2.25 
MAD: 1.11195 
Lower bound: -1.0858499999999998 
Upper bound: 5.58585
Outlier Indices: [191, 461, 1525] 
 ---------
Var

In [16]:
#Univariate Outliers - Bulimia Nervosa
BN_uni_vars = [None]*len(variables)
for i, x in enumerate(variables): #creates two variables for loop (i = 0, 1, 2.....)
    BN_uni_vars[i] = uni_outliers(df_ED_BN[x], x, threshold=3)
    print('Outlier Indices:', BN_uni_vars[i], '\n ---------')

Variable: Age
Median: 25.0 
MAD: 5.9304 
Lower bound: 7.2088 
Upper bound: 42.7912
Outlier Indices: [187, 1117, 1231] 
 ---------
Variable: BMI
Median: 22.318339100346023 
MAD: 3.9525423923058414 
Lower bound: 10.460711923428498 
Upper bound: 34.17596627726355
Outlier Indices: [187, 614, 1231, 1289] 
 ---------
Variable: YPSQ_EFSA
Median: 2.666666666666667 
MAD: 1.7297000000000002 
Lower bound: -2.5224333333333337 
Upper bound: 7.855766666666668
Outlier Indices: [] 
 ---------
Variable: YPSQ_S
Median: 4.0 
MAD: 1.4826 
Lower bound: -0.4478 
Upper bound: 8.4478
Outlier Indices: [] 
 ---------
Variable: YPSQ_EC
Median: 4.333333333333333 
MAD: 0.9883999999999991 
Lower bound: 1.3681333333333359 
Upper bound: 7.29853333333333
Outlier Indices: [1419] 
 ---------
Variable: YPSQ_O
Median: 2.5 
MAD: 1.4826 
Lower bound: -1.9478 
Upper bound: 6.9478
Outlier Indices: [] 
 ---------
Variable: YPSQ_EOS
Median: 3.0 
MAD: 1.4826 
Lower bound: -1.4478 
Upper bound: 7.4478
Outlier Indices: [] 
 ------

In [17]:
#Univariate Outliers - Binge Eating Disorder
BED_uni_vars = [None]*len(variables)
for i, x in enumerate(variables): #creates two variables for loop (i = 0, 1, 2.....)
    BED_uni_vars[i] = uni_outliers(df_ED_BED[x], x, threshold=3)
    print('Outlier Indices:', BED_uni_vars[i], '\n ---------')

Variable: Age
Median: 28.0 
MAD: 8.8956 
Lower bound: 1.313200000000002 
Upper bound: 54.6868
Outlier Indices: [] 
 ---------
Variable: BMI
Median: 28.67262607522348 
MAD: 6.907738555018983 
Lower bound: 7.949410410166532 
Upper bound: 49.39584174028043
Outlier Indices: [396, 1119, 1451] 
 ---------
Variable: YPSQ_EFSA
Median: 3.666666666666667 
MAD: 1.4826 
Lower bound: -0.781133333333333 
Upper bound: 8.114466666666667
Outlier Indices: [] 
 ---------
Variable: YPSQ_S
Median: 4.2 
MAD: 1.1860800000000002 
Lower bound: 0.6417599999999997 
Upper bound: 7.758240000000001
Outlier Indices: [] 
 ---------
Variable: YPSQ_EC
Median: 4.666666666666667 
MAD: 0.9883999999999991 
Lower bound: 1.7014666666666698 
Upper bound: 7.631866666666664
Outlier Indices: [66] 
 ---------
Variable: YPSQ_O
Median: 3.25 
MAD: 1.4826 
Lower bound: -1.1978 
Upper bound: 7.6978
Outlier Indices: [] 
 ---------
Variable: YPSQ_EOS
Median: 3.75 
MAD: 1.4826 
Lower bound: -0.6978 
Upper bound: 8.1978
Outlier Indices: [

In [18]:
#Univariate Outliers - OSFED
OSFED_uni_vars = [None]*len(variables)
for i, x in enumerate(variables): #creates two variables for loop (i = 0, 1, 2.....)
    OSFED_uni_vars[i] = uni_outliers(df_ED_OSFED[x], x, threshold=3)
    print('Outlier Indices:', OSFED_uni_vars[i], '\n ---------')

Variable: Age
Median: 25.0 
MAD: 4.4478 
Lower bound: 11.656600000000001 
Upper bound: 38.3434
Outlier Indices: [778, 1072, 1393] 
 ---------
Variable: BMI
Median: 23.307841210810572 
MAD: 5.460701547587363 
Lower bound: 6.925736568048482 
Upper bound: 39.68994585357267
Outlier Indices: [50, 571, 596, 758, 963, 1253] 
 ---------
Variable: YPSQ_EFSA
Median: 2.833333333333333 
MAD: 1.3590500000000003 
Lower bound: -1.2438166666666683 
Upper bound: 6.910483333333334
Outlier Indices: [] 
 ---------
Variable: YPSQ_S
Median: 4.0 
MAD: 1.4826 
Lower bound: -0.4478 
Upper bound: 8.4478
Outlier Indices: [] 
 ---------
Variable: YPSQ_EC
Median: 4.333333333333333 
MAD: 0.9884000000000004 
Lower bound: 1.3681333333333319 
Upper bound: 7.298533333333334
Outlier Indices: [637, 1019] 
 ---------
Variable: YPSQ_O
Median: 2.5 
MAD: 1.11195 
Lower bound: -0.8358499999999998 
Upper bound: 5.83585
Outlier Indices: [] 
 ---------
Variable: YPSQ_EOS
Median: 3.25 
MAD: 1.4826 
Lower bound: -1.1978 
Upper bou

In [19]:
#Univariate Outliers - Healthy
healthy_uni_vars = [None]*len(variables)
for i, x in enumerate(variables): #creates two variables for loop (i = 0, 1, 2.....)
    healthy_uni_vars[i] = uni_outliers(df_healthy[x], x, threshold=3)
    print('Outlier Indices:', healthy_uni_vars[i], '\n ---------')

Variable: Age
Median: 25.0 
MAD: 8.8956 
Lower bound: -1.686799999999998 
Upper bound: 51.6868
Outlier Indices: [5, 36, 59, 86, 96, 98, 109, 134, 147, 154, 196, 249, 258, 282, 287, 289, 293, 346, 353, 382, 384, 387, 411, 446, 481, 486, 549, 624, 749, 750, 906, 957, 965, 968, 1168, 1171, 1173, 1180, 1190, 1191, 1194, 1427, 1429, 1430, 1431, 1448, 1483, 1489, 1495, 1540, 1542, 1571] 
 ---------
Variable: BMI
Median: 22.779673527896495 
MAD: 5.094284202718979 
Lower bound: 7.496820919739557 
Upper bound: 38.062526136053435
Outlier Indices: [149, 215, 257, 347, 353, 370, 382, 411, 451, 516, 545, 548, 568, 749, 779, 872, 906, 925, 943, 952, 969, 1172, 1175, 1190, 1297, 1312, 1427, 1440, 1535, 1550, 1570] 
 ---------
Variable: YPSQ_EFSA
Median: 3.9166666666666665 
MAD: 1.6061500000000002 
Lower bound: -0.9017833333333338 
Upper bound: 8.735116666666666
Outlier Indices: [] 
 ---------
Variable: YPSQ_S
Median: 4.6 
MAD: 1.1860799999999996 
Lower bound: 1.041760000000001 
Upper bound: 8.15824
O

## Univariate Outlier Decisions
- Whilst outliers were detected, values for age and all scales were within the range of possible responses and none were non-sensical (i.e. Age > 125, or scale outside range). 
- Outliers were flagged in "Outliers.csv" and an analysis with all outlier values removed may be valuable. 
- Univariate outliers for BMI were removed at this stage, whilst others were maintained.


In [ ]:
#Univariate outliers dropped listwise from dataframe

# Pre-deletion shape check
print(df_ED_AN.shape)
print(df_ED_BN.shape)
print(df_ED_BED.shape)
print(df_ED_OSFED.shape)
print(df_healthy.shape)

# Outliers specified and removed
AN_univariate = ([471, 580, 883, 1066, 1094, 1126, 1236, 1262, 1438, 1454, 1549]) #11 
BN_univariate = ([187, 614, 1231, 1289]) #4 
BED_univariate = ([396, 1119, 1451]) #3
OSFED_univariate = ([50, 571, 596, 758, 963, 1253]) #6
healthy_univariate = ([149, 215, 257, 347, 353, 370, 382, 411, 451, 516, 545, 548, 568, 749, 779, 872, 906, 925, 943, 952, 969, 1172, 1175, 1190, 1297, 1312, 1427, 1440, 1535, 1550, 1570]) #31

df_ED_AN_UOR = df_ED_AN.drop(AN_univariate)
df_ED_BN_UOR = df_ED_BN.drop(BN_univariate)
df_ED_BED_UOR = df_ED_BED.drop(BED_univariate)
df_ED_OSFED_UOR = df_ED_OSFED.drop(OSFED_univariate)
df_healthy_UOR = df_healthy.drop(healthy_univariate)

# Deletion confirmed with shape check
print(df_ED_AN_UOR.shape)
print(df_ED_BN_UOR.shape)
print(df_ED_BED_UOR.shape)
print(df_ED_OSFED_UOR.shape)
print(df_healthy_UOR.shape)

#Anorexia Nervosa
df_ED_AN_UOR.to_csv('AnorexiaDataset_R_MVOutlier_Removal.csv', index=True) #save dataset
#Bulimia Nervosa
df_ED_BN_UOR.to_csv('BulimiaDataset_R_MVOutlier_Removal.csv', index=True) #save dataset
#Binge Eating Disorder
df_ED_BED_UOR.to_csv('BingeEatingDataset_R_MVOutlier_Removal.csv', index=True) #save dataset
#OSFED
df_ED_OSFED_UOR.to_csv('OsfedDataset_R_MVOutlier_Removal.csv', index=True) #save dataset
#Healthy
df_healthy_UOR.to_csv('HealthyDataset_R_MVOutlier_Removal.csv', index=True) #save dataset

## Multivariate outlier analysis
- MV outlier analysis conducted in R-studio using Mahalanobis Minimum Covariant Determinant variant from Leys et al. (2018)
- MV outliers assessed on a case by case basis.
    - AN = 5 ([934, 1078, 1208, 1295, 1525])
    - BN = 15 ([308, 367, 378, 570, 609, 673, 739, 782, 846,1030, 1112, 1117, 1274, 1418, 1419])
    - BED = 11 ([166, 369, 436, 447, 449, 700, 857, 893, 979, 1467, 1515])
    - OSFED = 7 ([35, 291, 572, 708, 1019, 1393, 1520])
    - Healthy = 15 ([41, 71, 135, 328, 430, 827, 839, 869, 887, 900, 1000, 1029, 1037, 1180, 1410])
- Outliers kept for descriptives but removed for correlations and HMR analysis

In [21]:
# multivariate outliers removed
AN_multivariate = ([934, 1078, 1208, 1295, 1525])
BN_multivariate = ([308, 367, 378, 570, 609, 673, 739, 782, 846,1030, 1112, 1117, 1274, 1418, 1419])
BED_multivariate = ([166, 369, 436, 447, 449, 700, 857, 893, 979, 1467, 1515])
OSFED_multivariate = ([35, 291, 572, 708, 1019, 1393, 1520])
healthy_multivariate = ([41, 71, 135, 328, 430, 827, 839, 869, 887, 900, 1000, 1029, 1037, 1180, 1410])

# drop multivariate outliers
df_ED_AN_UMR = df_ED_AN_UOR.drop(AN_multivariate)
df_ED_BN_UMR = df_ED_BN_UOR.drop(BN_multivariate)
df_ED_BED_UMR = df_ED_BED_UOR.drop(BED_multivariate)
df_ED_OSFED_UMR = df_ED_OSFED_UOR.drop(OSFED_multivariate)
df_healthy_UMR = df_healthy_UOR.drop(healthy_multivariate)

In [23]:
 # Participants with sex == 3 (not disclosed) removed prior to saving CSV file 

#print number of indexes where sex was reported as non-disclosed
print(df_ED_AN_UMR['Sex'].value_counts()[3])
print(df_ED_BN_UMR['Sex'].value_counts()[3])
print(df_ED_OSFED_UMR['Sex'].value_counts()[3])
print(df_healthy_UMR['Sex'].value_counts()[3])

#Drop where Sex = 3 (not disclosed) as this will impact regression
df_ED_AN_UMR.drop(df_ED_AN_UMR.loc[df_ED_AN_UMR['Sex']==3].index, inplace=True)
df_ED_BN_UMR.drop(df_ED_BN_UMR.loc[df_ED_BN_UMR['Sex']==3].index, inplace=True)
df_ED_BED_UMR.drop(df_ED_BED_UMR.loc[df_ED_BED_UMR['Sex']==3].index, inplace=True)
df_ED_OSFED_UMR.drop(df_ED_OSFED_UMR.loc[df_ED_OSFED_UMR['Sex']==3].index, inplace=True)
df_healthy_UMR.drop(df_healthy_UMR.loc[df_healthy_UMR['Sex']==3].index, inplace=True)


3
2
27
2
7


In [24]:
#Checks number of cases after outlier and undisclosed sex removal
print("Number of AN cases remaining for inferential analysis = " + str(len(df_ED_AN_UMR['Sex'])))
print("Number of BN cases remaining for inferential analysis = " + str(len(df_ED_BN_UMR['Sex'])))
print("Number of BED cases remaining for inferential analysis = " + str(len(df_ED_BED_UMR['Sex'])))
print("Number of OSFED cases remaining for inferential analysis = " + str(len(df_ED_OSFED_UMR['Sex'])))
print("Number of healthy cases remaining for inferential analysis = " + str(len(df_healthy_UMR['Sex'])))

Number of AN cases remaining for inferential analysis = 155
Number of BN cases remaining for inferential analysis = 55
Number of BED cases remaining for inferential analysis = 33
Number of OSFED cases remaining for inferential analysis = 93
Number of healthy cases remaining for inferential analysis = 505


In [ ]:
# Data (with univariate and multivariate outlier removed) exported to csv files and saved in clean folder. 
os.chdir('c:\\Users\\j_m289\\Pictures\\PHD\\15. Data Analysis\HMR\Clean')  # Directory set to 'clean' folder

# Take columns from dataset needed for analysis and assign to new dataframe
AN_analysis_dataset = df_ED_AN_UMR[['ID','Age','Height','Weight','BMI','Sex','Education','Nationality','MH_Diag','EatD','MDD','BiP','SAD','GAD','PaD','PTSD','SUD','OCD','BDD','BPD','ADHD','Other','MH_ND',\
    'MHBurden','ED_Diag','ED_subtype','past_ED_Diag','pastAN','pastBN','pastBED','partOSFED','ED_treatment','Appearance_Satisfact','YPSQ_EFSA','YPSQ_S','YPSQ_EC','YPSQ_O','YPSQ_EOS',\
    'YPSQ_SCRE','YPSQ_DS','YPSQ_SB','YPSQ_HSC','EmReg_CR','EmReg_ES','EDE_Q_R','EDE_Q_W','EDE_Q_S','EDE_Q_E', 'EDE_Q_G','CFI_Alt', 'CFI_Ctrl']]

BN_analysis_dataset = df_ED_BN_UMR[['ID','Age','Height','Weight','BMI','Sex','Education','Nationality','MH_Diag','EatD','MDD','BiP','SAD','GAD','PaD','PTSD','SUD','OCD','BDD','BPD','ADHD','Other','MH_ND',\
    'MHBurden','ED_Diag','ED_subtype','past_ED_Diag','pastAN','pastBN','pastBED','partOSFED','ED_treatment','Appearance_Satisfact','YPSQ_EFSA','YPSQ_S','YPSQ_EC','YPSQ_O','YPSQ_EOS',\
    'YPSQ_SCRE','YPSQ_DS','YPSQ_SB','YPSQ_HSC','EmReg_CR','EmReg_ES','EDE_Q_R','EDE_Q_W','EDE_Q_S','EDE_Q_E', 'EDE_Q_G','CFI_Alt', 'CFI_Ctrl']]

BED_analysis_dataset = df_ED_BED_UMR[['ID','Age','Height','Weight','BMI','Sex','Education','Nationality','MH_Diag','EatD','MDD','BiP','SAD','GAD','PaD','PTSD','SUD','OCD','BDD','BPD','ADHD','Other','MH_ND',\
    'MHBurden','ED_Diag','ED_subtype','past_ED_Diag','pastAN','pastBN','pastBED','partOSFED','ED_treatment','Appearance_Satisfact','YPSQ_EFSA','YPSQ_S','YPSQ_EC','YPSQ_O','YPSQ_EOS',\
    'YPSQ_SCRE','YPSQ_DS','YPSQ_SB','YPSQ_HSC','EmReg_CR','EmReg_ES','EDE_Q_R','EDE_Q_W','EDE_Q_S','EDE_Q_E', 'EDE_Q_G','CFI_Alt', 'CFI_Ctrl']]

OSFED_analysis_dataset = df_ED_OSFED_UMR[['ID','Age','Height','Weight','BMI','Sex','Education','Nationality','MH_Diag','EatD','MDD','BiP','SAD','GAD','PaD','PTSD','SUD','OCD','BDD','BPD','ADHD','Other','MH_ND',\
    'MHBurden','ED_Diag','ED_subtype','past_ED_Diag','pastAN','pastBN','pastBED','partOSFED','ED_treatment','Appearance_Satisfact','YPSQ_EFSA','YPSQ_S','YPSQ_EC','YPSQ_O','YPSQ_EOS',\
    'YPSQ_SCRE','YPSQ_DS','YPSQ_SB','YPSQ_HSC','EmReg_CR','EmReg_ES','EDE_Q_R','EDE_Q_W','EDE_Q_S','EDE_Q_E', 'EDE_Q_G','CFI_Alt', 'CFI_Ctrl']]

Healthy_analysis_dataset = df_healthy_UMR[['ID','Age','Height','Weight','BMI','Sex','Education','Nationality','MH_Diag','EatD','MDD','BiP','SAD','GAD','PaD','PTSD','SUD','OCD','BDD','BPD','ADHD','Other','MH_ND',\
    'MHBurden','ED_Diag','ED_subtype','past_ED_Diag','pastAN','pastBN','pastBED','partOSFED','ED_treatment','Appearance_Satisfact','YPSQ_EFSA','YPSQ_S','YPSQ_EC','YPSQ_O','YPSQ_EOS',\
    'YPSQ_SCRE','YPSQ_DS','YPSQ_SB','YPSQ_HSC','EmReg_CR','EmReg_ES','EDE_Q_R','EDE_Q_W','EDE_Q_S','EDE_Q_E', 'EDE_Q_G','CFI_Alt', 'CFI_Ctrl']]

#round values to 2 decimal places (pass variables for columns)

AN_analysis_dataset = AN_analysis_dataset.round(2)
BN_analysis_dataset = BN_analysis_dataset.round(2)
BED_analysis_dataset = BED_analysis_dataset.round(2)
OSFED_analysis_dataset = OSFED_analysis_dataset.round(2)
Healthy_analysis_dataset = Healthy_analysis_dataset.round(2)

# Export dataframes to csv file for analysis in SPSS.
#Anorexia Nervosa
AN_analysis_dataset.to_csv('AnorexiaDataset_Final.csv', index=True) #save dataset
#Bulimia Nervosa
BN_analysis_dataset.to_csv('BulimiaDataset_Final.csv', index=True) #save dataset
#Binge Eating Disorder
BED_analysis_dataset.to_csv('BingeEatingDataset_Final.csv', index=True) #save dataset
#OSFED
OSFED_analysis_dataset.to_csv('OsfedDataset_Final.csv', index=True) #save dataset
#Healthy
Healthy_analysis_dataset.to_csv('HealthyDataset_Final.csv', index=True) #save dataset